In [2]:
import sqlite3

conexion = sqlite3.connect("bd_evaluacion.db")
conexion.commit()
conexion.close()


In [3]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS autores (
    id_autor INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_autor TEXT UNIQUE NOT NULL)
    """)
conexion.commit()
conexion.close()

In [4]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS categoria (
id_categoria INTEGER PRIMARY KEY AUTOINCREMENT,
nombre_categoria TEXT UNIQUE NOT NULL)
""")
conexion.commit()
conexion.close()

In [5]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS libro_autor (
    id_libro INTEGER NOT NULL,
    id_autor INTEGER NOT NULL,
    PRIMARY KEY (id_libro, id_autor),
    FOREIGN KEY (id_libro) REFERENCES libros(id_libros),
    FOREIGN KEY (id_autor) REFERENCES autores(id_autor)
)
""")
conexion.commit()
conexion.close()

In [ ]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS libros (
id_libros INTEGER PRIMARY KEY AUTOINCREMENT,
titulo TEXT NOT NULL,
precio REAL NOT NULL,
calificacion INTEGER NOT NULL,
enlace TEXT NOT NULL,
id_categoria INTEGER NOT NULL,
FOREIGN KEY (id_categoria) REFERENCES categoria(id_categoria)
)""")
conexion.commit()
conexion.close()

In [ ]:
import requests
from bs4 import BeautifulSoup
import time 

web = "https://books.toscrape.com/"

def convertir_calificacion(texto):
    numero = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }
    return numero.get(texto, 0) 

conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()


def obtener_autores(titulo):
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{titulo}"

    try:
        respuesta = requests.get(url, timeout=10)
        autor = respuesta.json()

        if "items" in autor and len(autor["items"]) > 0:
            info = autor["items"][0].get("volumeInfo", {})
            autores = info.get("authors", [])
            if autores:
                return autores
            else:
                return "Autor desconocido"
        
    except Exception as e:
        print(f"error al conseguir autor de {titulo} {e}")
        return "error al buscar"

def obtener_libros():
    for pagina in range(1, 51):
        url = f"https://books.toscrape.com/catalogue/page-{pagina}.html"
        respuesta = requests.get(url)
        soup = BeautifulSoup(respuesta.text, "html.parser")
        links = soup.select("h3 a")

        
        for link in links:
            url_links = link['href']
            url_completa = web + "catalogue/" + url_links.replace('../', '')

            try:
                respuesta_libro = requests.get(url_completa, timeout=10)
                soup_libro = BeautifulSoup(respuesta_libro.text, 'html.parser')

                titulo = soup_libro.h1.text 

                precio_libro = soup_libro.select_one('.product_main .price_color')
                precio_simbolos = precio_libro.text 
                precio = float(precio_simbolos.replace("Â£", "").strip())

                categoria_libro = soup_libro.select('ul.breadcrumb li')
                categoria = categoria_libro[2].text.strip()

                calificacion_libro = soup_libro.select_one('.product_main .star-rating')
                calificacion_texto = calificacion_libro['class'][1]
                calificacion = convertir_calificacion(calificacion_texto)

                autores = obtener_autores(titulo)
                time.sleep(1)

                cursor.execute("INSERT OR IGNORE INTO categoria (nombre_categoria) VALUES (?)",(categoria,))
                cursor.execute("SELECT id_categoria FROM categoria WHERE nombre_categoria = ?", (categoria,))
                id_categoria = cursor.fetchone()[0]

                cursor.execute("INSERT INTO libros (titulo, precio, calificacion, enlace, id_categoria) VALUES (?, ?, ?, ?, ?)", (titulo, precio, calificacion, url_completa, id_categoria))
                id_libro = cursor.lastrowid

                for autor in autores:
                    cursor.execute("INSERT OR IGNORE INTO autores (nombre_autor) VALUES (?)",(autor,))
                    cursor.execute("SELECT id_autor FROM autores WHERE nombre_autor = ?", (autor,))
                    id_autor = cursor.fetchone()[0]

                    cursor.execute("INSERT OR IGNORE INTO libro_autor (id_libro, id_autor) VALUES (?, ?)", (id_libro, id_autor))
                            
            except Exception as e:
                print(f"error procesando libro: {url_completa} {e}")
        
    conexion.commit()
    conexion.close()

if __name__ == "__main__":
    obtener_libros()

In [8]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()

cursor.execute("""
    SELECT libros.titulo, categoria.nombre_categoria
    FROM libros
    JOIN categoria ON libros.id_categoria = categoria.id_categoria 
    LIMIT  3;
""")

resultados = cursor.fetchall()

for fila in resultados:
    print(f"titulo: {fila[0]} | categoria: {fila[1]}")

conexion.close()

titulo: A Light in the Attic | categoria: Poetry
titulo: Tipping the Velvet | categoria: Historical Fiction
titulo: Soumission | categoria: Fiction


In [9]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()

cursor.execute("""
    SELECT libros.titulo, autores.nombre_autor, libros.precio, libros.calificacion, categoria.nombre_categoria
    FROM libros
    JOIN libro_autor ON libros.id_libros = libro_autor.id_libro
    JOIN autores ON libro_autor.id_autor = autores.id_autor
    JOIN categoria ON libros.id_categoria = categoria.id_categoria
    WHERE libros.calificacion = 1 
    LIMIT 5;
""")
resultados = cursor.fetchall()

for fila in resultados:
    print(f"titulo: {fila[0]} | autor: {fila[1]} | precio: {fila[2]} | calificación: {fila[3]} | categoria: {fila[4]}")

conexion.close()

titulo: Tipping the Velvet | autor: Sarah Waters | precio: 53.74 | calificación: 1 | categoria: Historical Fiction
titulo: Soumission | autor: Michel Houellebecq | precio: 50.1 | calificación: 1 | categoria: Fiction
titulo: The Requiem Red | autor: Edward Komara | precio: 22.65 | calificación: 1 | categoria: Young Adult
titulo: The Requiem Red | autor: Peter Lee | precio: 22.65 | calificación: 1 | categoria: Young Adult
titulo: The Black Maria | autor: Aracelis Girmay | precio: 52.15 | calificación: 1 | categoria: Poetry


In [11]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()
cursor.execute("""
    SELECT libros.titulo, COUNT(*)
    FROM libros
    JOIN libro_autor ON libros.id_libros = libro_autor.id_libro
    GROUP BY libros.id_libros
    HAVING COUNT(*) = 2
    LIMIT 5;
""")
resultado = cursor.fetchall()

for fila in resultado:
    print(f"libro: {fila[0]} | cantidad de autores: {fila[1]}")

conexion.close()

libro: The Requiem Red | cantidad de autores: 2
libro: The Four Agreements: A Practical Guide to Personal Freedom | cantidad de autores: 2
libro: The Mindfulness and Acceptance Workbook for Anxiety: A Guide to Breaking Free from Anxiety, Phobias, and Worry Using Acceptance and Commitment Therapy | cantidad de autores: 2
libro: Security | cantidad de autores: 2
libro: Reskilling America: Learning to Labor in the Twenty-First Century | cantidad de autores: 2


In [14]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()

cursor.execute("""
    SELECT libros.titulo, autores.nombre_autor
    FROM libros
    JOIN libro_autor ON libros.id_libros = libro_autor.id_libro
    JOIN autores ON libro_autor.id_autor = autores.id_autor
    WHERE libros.titulo = ?
""", ("The Requiem Red",))
autores = cursor.fetchall()

for fila in autores:
    print(f"titulo: {fila[0]} | autor: {fila[1]}")

conexion.close()

titulo: The Requiem Red | autor: Edward Komara
titulo: The Requiem Red | autor: Peter Lee


In [15]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()

cursor.execute("""
    CREATE INDEX IF NOT EXISTS idx_autores_nombre ON autores(nombre_autor);""")
conexion.commit()
conexion.close()

In [17]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()

cursor.execute("""
    SELECT libros.titulo
    FROM libros
    JOIN libro_autor ON libros.id_libros = libro_autor.id_libro
    JOIN autores ON libro_autor.id_autor = autores.id_autor
    WHERE autores.nombre_autor = ?;
""", ("Aracelis Girmay",))
respuesta = cursor.fetchall()

for fila in respuesta:
    print(f"libro: {fila[0]}")

conexion.close()

libro: The Black Maria


In [ ]:
conexion = sqlite3.connect("bd_evaluacion.db")
cursor = conexion.cursor()

cursor.execute("""
    SELECT libros.titulo, autores.nombre_autor
    FROM libros
    JOIN libro_autor ON libros.id_libros = libro_autor.id_libro
    JOIN autores ON libro_autor.id_autor = autores.id_autor
    WHERE autores.nombre_autor LIKE ?;
""", ("%Robert%",))
respuesta = cursor.fetchall()

for fila in respuesta:
    print(f"libro: {fila[0]} | autor: {fila[1]}")

conexion.close()

libro: Outcast, Vol. 1: A Darkness Surrounds Him (Outcast #1) | autor: Robert Kirkman
libro: The Wedding Pact (The O'Malleys #2) | autor: Katee Robert
libro: Feathers: Displays of Brilliant Plumage | autor: Robert Clark
libro: The Bourne Identity (Jason Bourne #1) | autor: Robert Ludlum
libro: Rich Dad, Poor Dad | autor: Robert T. Kiyosaki
libro: Catherine the Great: Portrait of a Woman | autor: Robert K. Massie
libro: The Bone Hunters (Lexy Vaughan & Steven Macaulay #2) | autor: Robert J. Mrazek
libro: The Silkworm (Cormoran Strike #2) | autor: Robert Galbraith
libro: The Cuckoo's Calling (Cormoran Strike #1) | autor: Robert Galbraith
libro: Career of Evil (Cormoran Strike #3) | autor: Robert Galbraith
libro: The Girl In The Ice (DCI Erika Foster #1) | autor: Robert Bryndza


In [ ]:
# RELACION DE BASE DE DATOS 
# Tengo una relacion de 1:n con mi tabla libros y categoria 
# En la tabla libros y autores hago una relacion n:n atravez de una tabla intermedia libro_autor ya que un libro puede tener varios autores y un autor puede tener varios libros 
# ┌──────────────┐     ┌──────────────┐     ┌────────────────────────────────────────────────────────────┐     ┌────────────────────┐
# │   autores    │◄────┤  libro_autor │────►                      libros                                 │────►      categoria      │
# ├──────────────┤     ├──────────────┤     ├────────────────────────────────────────────────────────────┤     ├────────────────────┤
# │ id_autor (PK)│     │ id_autor (FK)│     │ id_libros (PK)                                             │     │ id_categoria (PK)  │
# │ nombre_autor │     │ id_libro (FK)│     │ titulo                                                     │     │ nombre_categoria   │
# └──────────────┘     └──────────────┘     │ precio                                                     │     └────────────────────┘
#                                           │ calificacion                                               │
#                                           │ enlace                                                     │
#                                           │ id_categoria (FK → categoria.id_categoria, NOT NULL)       │
#                                           └────────────────────────────────────────────────────────────┘
